In [61]:
import pandas as pd
from configurations import basic_parameters as bp

In [62]:
csv_dataframe = bp['dataframe_dir'] + '/' + bp['dataframe_file']

In [63]:
names = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']

df = pd.read_csv(csv_dataframe, names=names, usecols=['Open time', 'Open', 'High', 'Low', 'Close'])

In [64]:
def computeRSI (data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

In [65]:
def stochastic(data, k_window, d_window, window):
    
    # input to function is one column from df
    # containing closing price or whatever value we want to extract K and D from
    
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()
    
    stoch = ( (data - min_val) / (max_val - min_val) ) * 100
    
    K = stoch.rolling(window=k_window, center=False).mean() 
    #K = stoch
    
    D = K.rolling(window=d_window, center=False).mean() 


    return K, D

In [66]:
df['RSI'] = computeRSI(df['Close'], 14)
df['K'], df['D'] = stochastic(df['RSI'], 3, 3, 14)

In [67]:
df['time UTC'] = pd.to_datetime(df['Open time'], unit='ms', origin='unix')
df.dropna(inplace=True)
df

,Open time,Open,High,Low,Close,RSI,K,D,time UTC
31,1678060831000,22422.12,22423.83,22421.52,22422.54,54.033107,91.624977,95.381293,2023-03-06 00:00:31
32,1678060832000,22422.54,22424.50,22422.31,22422.54,54.033107,88.731052,91.624977,2023-03-06 00:00:32
33,1678060833000,22422.55,22422.73,22422.52,22422.55,54.080566,91.371062,90.575697,2023-03-06 00:00:33
34,1678060834000,22422.53,22423.00,22422.32,22422.32,52.732041,89.725047,89.942387,2023-03-06 00:00:34
35,1678060835000,22422.46,22423.01,22422.16,22422.16,51.765031,86.953478,89.349862,2023-03-06 00:00:35
...,...,...,...,...,...,...,...,...,...
86395,1678147195000,22408.90,22409.75,22408.41,22408.53,50.694848,6.975464,12.929896,2023-03-06 23:59:55
86396,1678147196000,22408.96,22409.84,22408.54,22409.72,58.654142,16.822415,10.257781,2023-03-06 23:59:56
86397,1678147197000,22409.28,22409.28,22408.89,22408.89,52.311174,16.042611,13.280163,2023-03-06 23:59:57
86398,1678147198000,22409.61,22409.73,22408.90,22408.90,52.377994,18.862486,17.242504,2023-03-06 23:59:58


In [68]:
stochastic_value = 0
RSI_value = 40
df.loc[(df['D'] <= stochastic_value) & (df['RSI'] <= RSI_value), 'buy?'] = True
#df.loc[df['D'] <= 5, 'buy?'] = True
#df.loc[df['buy?'] == True]

In [69]:
rolling_seconds = 60

df['min_val'] = df['Low'].shift(-rolling_seconds).rolling(window=rolling_seconds).min()
df['max_val'] = df['High'].shift(-rolling_seconds).rolling(window=rolling_seconds).max()

df['max_val_%'] = (df['max_val'] - df['Open'].shift(-1)) * 100 / df['Open'].shift(-1)
df['min_val_%'] = (df['min_val'] - df['Open'].shift(-1)) * 100 / df['Open'].shift(-1)

#df.dropna(inplace=True)
#df[60:70]
#df.loc[df['buy?'] == True]
df = df.loc[df['buy?'] == True]
df = df.loc[df['Open time'] - df['Open time'].shift(1) > rolling_seconds * 1000] # wait until trade 
df

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?,min_val,max_val,max_val_%,min_val_%
3207,1678064007000,22445.45,22446.47,22444.89,22445.16,19.767124,0.0,0.0,2023-03-06 00:53:27,True,22436.68,22449.09,0.017509,-0.037781
7298,1678068098000,22363.68,22364.89,22363.18,22363.27,19.470610,0.0,0.0,2023-03-06 02:01:38,True,22344.01,22365.33,0.003219,-0.092110
13184,1678073984000,22392.87,22392.87,22389.01,22390.00,10.843502,0.0,0.0,2023-03-06 03:39:44,True,22376.65,22390.56,0.006834,-0.055295
15853,1678076653000,22384.12,22384.22,22378.98,22379.00,9.430421,0.0,0.0,2023-03-06 04:24:13,True,22376.31,22386.65,0.031725,-0.014478
16315,1678077115000,22339.49,22340.65,22338.92,22339.07,17.266772,0.0,0.0,2023-03-06 04:31:55,True,22335.79,22349.41,0.043152,-0.017816
27795,1678088595000,22408.50,22408.64,22407.70,22408.18,15.249239,0.0,0.0,2023-03-06 07:43:15,True,22406.06,22410.70,0.011246,-0.009461
35368,1678096168000,22389.88,22390.15,22389.30,22389.30,15.146961,0.0,0.0,2023-03-06 09:49:28,True,22387.50,22390.70,0.004556,-0.009737
39815,1678100615000,22392.05,22392.48,22391.18,22391.57,14.345371,0.0,0.0,2023-03-06 11:03:35,True,22385.64,22392.44,0.003617,-0.026751
40098,1678100898000,22384.74,22385.00,22383.60,22384.51,16.534802,0.0,0.0,2023-03-06 11:08:18,True,22384.35,22390.46,0.026447,-0.000849
40577,1678101377000,22387.44,22387.64,22387.05,22387.43,18.845622,0.0,0.0,2023-03-06 11:16:17,True,22385.00,22388.48,0.004467,-0.011078


In [70]:
tp_size = 0.075
sl_size = 0.075
# 0.075 RSI 40 total profit 2.47

In [71]:
sl = df.loc[(df['buy?'] == True) & (df['min_val_%'] < -sl_size)]
#sl = sl.loc[sl['Open time'] - sl['Open time'].shift(1) > rolling_seconds * 1000]
sl

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?,min_val,max_val,max_val_%,min_val_%
7298,1678068098000,22363.68,22364.89,22363.18,22363.27,19.470610,0.0,0.0,2023-03-06 02:01:38,True,22344.01,22365.33,0.003219,-0.092110
69465,1678130265000,22453.40,22454.64,22443.88,22450.45,11.523151,0.0,0.0,2023-03-06 19:17:45,True,22370.00,22463.95,0.060132,-0.358345


In [72]:
tp = df.loc[(df['buy?'] == True) & (df['max_val_%'] > tp_size)]
#tp = tp.loc[tp['Open time'] - tp['Open time'].shift(1) >  rolling_seconds * 1000]
tp[:50]

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?,min_val,max_val,max_val_%,min_val_%
69673,1678130473000,22377.19,22377.79,22373.35,22375.32,19.26339,0.0,0.0,2023-03-06 19:21:13,True,22369.0,22407.18,0.141897,-0.028737


In [73]:
tp.shape[0]/sl.shape[0]

0.5

In [74]:
(tp.shape[0] * tp_size) - (sl.shape[0] * sl_size)

-0.075